In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

In [5]:
from scipy import stats
from scipy.stats import norm, gamma, linregress, skew
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering 
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, confusion_matrix, silhouette_score, silhouette_samples

In [6]:
def dump_pickle(file, filename):
    outfile = open(filename, 'wb')
    pickle.dump(file, outfile)
    outfile.close()

def load_pickle(filename):
    infile = open(filename, 'rb')
    file = pickle.load(infile)
    infile.close()
    return file

In [7]:
sns.set()

## Data Prep

Read cluster data

In [8]:
df = pd.read_csv('nba_cleaned.csv')
df.shape

(1774, 116)

In [9]:
label_cols = ['Player', 'Season', 'Team', 'player_cluster']
label_df = df[label_cols].rename(columns={'Season': 'season',
                                          'Team': 'team'})
label_df.head()

,Player,season,team,player_cluster
0,Marc Gasol,2017,Memphis Grizzlies,6
1,Tyreke Evans,2017,Memphis Grizzlies,2
2,Dillon Brooks,2017,Memphis Grizzlies,1
3,Jarell Martin,2017,Memphis Grizzlies,3
4,Mario Chalmers,2017,Memphis Grizzlies,1


Read lineup data by CleaningTheGlass

In [10]:
lineup_df = pd.read_csv('ctg_nba_lineup_stats.csv')
lineup_df = lineup_df[lineup_df['season'] != 2016].reset_index(drop=True)
lineup_df.shape

(67655, 30)

In [11]:
lineup_cols = ['PG', 'SG', 'SF', 'PF', 'C', 'season', 'team', 'Poss'] + \
              [col for col in lineup_df.columns if 'off_' in col]
lineup_df = lineup_df[lineup_cols]
lineup_df.head()

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr
0,Dennis Schroder,Kent Bazemore,Taurean Prince,Ersan Ilyasova,Miles Plumlee,2017,Atlanta Hawks,733,18.0,99.9,41.0,51.5%,12.0,17.7%,59.0,25.9%,37.0,15.0
1,Dennis Schroder,Kent Bazemore,Taurean Prince,John Collins,Dewayne Dedmon,2017,Atlanta Hawks,245,66.0,112.7,81.0,57.9%,15.0,17.1%,37.0,23.1%,41.0,15.7
2,Dennis Schroder,Kent Bazemore,Taurean Prince,Ersan Ilyasova,Dewayne Dedmon,2017,Atlanta Hawks,193,22.0,101.0,18.0,48.8%,78.0,12.4%,22.0,20.4%,73.0,22.1
3,Dennis Schroder,Kent Bazemore,Taurean Prince,Luke Babbitt,Dewayne Dedmon,2017,Atlanta Hawks,142,62.0,112.0,91.0,59.7%,7.0,19.0%,44.0,23.8%,5.0,8.9
4,Isaiah Taylor,Taurean Prince,Damion Lee,John Collins,Dewayne Dedmon,2017,Atlanta Hawks,140,36.0,105.7,41.0,51.5%,62.0,13.6%,36.0,23.1%,4.0,8.3


Read kmeans models

In [12]:
nba_clustering_models_km = load_pickle('nba_clustering_models_km')
pre_pca_pipe = nba_clustering_models_km['pre_pca_pipe']
pca = nba_clustering_models_km['pca']
pre_cluster_scaler = nba_clustering_models_km['pre_cluster_scaler']
cluster_map_km_to_sc = nba_clustering_models_km['cluster_map_to_spectral_clustering']
km = nba_clustering_models_km['km']

In [13]:
cluster_dict_detail = {2: 'Ballhandler (Iso+PnR)', 1: 'Ballhandler (PnR)',
                       6: 'Big (Post Up+Roll)', 5: 'Big (Roll+Cut)',
                       0: 'Wing (Off Screen)', 4: 'Wing (Hand Off)', 
                       3: 'Wing (Spot Up+Transition)'}

Read Synergy data

In [14]:
s_df = pd.read_csv('nba_offense.csv')
s_df = s_df[s_df['Season'] != 2016]
s_df['Player'] = s_df['Player'].apply(lambda x: ' '.join(x.split(' ')[1:]))
s_df['Overall_Poss'] = s_df['Overall_Poss'] - s_df['Miscellaneous_Poss']

In [15]:
poss_weights_cols = ['Spot Up_Poss%', 'P&R Ball Handler_Poss%', 
                     'Hand Off_Poss%', 'Off Screen_Poss%',
                     'Cut_Poss%', 'Offensive Rebounds (put backs)_Poss%',
                     'P&R Roll Man_Poss%', 'Isolation_Poss%', 
                     'Post-Up_Poss%', 'Transition_Poss%'] 

In [16]:
for col in poss_weights_cols:
    s_df[col] = s_df[col.replace('%', '')] / s_df['Overall_Poss']
    s_df[col] = s_df[col].fillna(0)

Synergy functions

In [17]:
def player_lookup(player, df, season=2021, if_ppp_n_rank=False):
    pdf = df[(df['Player'] == player) & (df['Season'] == season)]
    for idx in pdf.index:
        print(player, pdf.loc[idx]['Team'])
        print('-')
        for col in df.columns:
            if 'Poss%' in col and 'Miscellaneous' not in col:
                print(col, f'{pdf.loc[idx][col]:.2%}')
                if if_ppp_n_rank:
                    ppp_col = col.replace('Poss%', 'PPP')
                    print(ppp_col, f'{pdf.loc[idx][ppp_col]:.3f}')
                    rank_col = col.replace('Poss%', 'Rank')
                    print(rank_col, f'{pdf.loc[idx][rank_col]}')
                print('-')

## Feature Engineering

### Get League/Team Offensive Rating for each Season

In [18]:
tm_lineup_df = lineup_df.copy()
tm_lineup_df['off_Pts'] = lineup_df['off_Pts/Poss'] * lineup_df['Poss']

In [19]:
season_off_ratings = tm_lineup_df.groupby('season')[['off_Pts', 'Poss']].sum()
season_off_ratings['off_Pts/Poss'] = season_off_ratings['off_Pts'] / \
                                     season_off_ratings['Poss']
season_off_ratings[['off_Pts/Poss']].reset_index()

,season,off_Pts/Poss
0,2017,108.329988
1,2018,110.480204
2,2019,110.793898
3,2020,112.795304
4,2021,112.255389


In [20]:
tm_off_ratings = tm_lineup_df.groupby(['team', 'season'])[['off_Pts', 'Poss']].sum()
tm_off_ratings['off_Pts/Poss'] = tm_off_ratings['off_Pts'] / tm_off_ratings['Poss']
tm_off_ratings = tm_off_ratings[['off_Pts/Poss']].reset_index()\
                                                 .rename(columns={'off_Pts/Poss': 'tm_off_Pts/Poss'})
tm_off_ratings

,team,season,tm_off_Pts/Poss
0,Atlanta Hawks,2017,104.452049
1,Atlanta Hawks,2018,107.673913
2,Atlanta Hawks,2019,107.546858
3,Atlanta Hawks,2020,115.375155
4,Atlanta Hawks,2021,116.125381
...,...,...,...
145,Washington Wizards,2017,108.547292
146,Washington Wizards,2018,110.997148
147,Washington Wizards,2019,110.553583
148,Washington Wizards,2020,111.587056


### Merge

### Step 1: Find out who did not have a cluster tag

In [21]:
synergy_name_dict = {'kevinporterjr': 'kevinporter',
                     'patrickbeverly': 'patrickbeverley',
                     'terryrozieriii': 'terryrozier',
                     'kellyoubrejr': 'kellyoubre',
                     'lonniewalkeriv': 'lonniewalker',
                     'ottoporterjr': 'ottoporter',
                     'robertwilliamsiii': 'robertwilliams',
                     'wendellcarterjr': 'wendellcarter',
                     'alfarouqaminu': 'al-farouqaminu',
                     'jarenjacksonjr': 'jarenjackson',
                     'marvinbagleyiii': 'marvinbagley',
                     'marcusmorrissr': 'marcusmorris',
                     'wesleyiwundu': 'wesiwundu',
                     'troybrownjr': 'troybrown',
                     'larrynance': 'larrynancejr',
                     'michaelporterjr': 'michaelporter',
                     'kevinknoxii': 'kevinknox',
                     'jamesennisiii': 'jamesennis',
                     'danilogalinari': 'danilogallinari',
                     'danuelhousejr': 'danuelhouse',
                     'svimykhailiuk': 'sviatoslavmykhailiuk',
                     'timothéluwawu-cabarrot': 'timotheluwawu-cabarrot',
                     'duanewashingtonjr': 'duanewashington',
                     'juanchohernangomez': 'juanhernangomez',
                     'harrygilesiii': 'harrygiles',
                     'malcomdelaney': 'malcolmdelaney'}

In [22]:
label_df['player_alt'] = label_df['Player'].apply(lambda x: x.replace(' ', '').lower())\
                                           .apply(lambda x: x.replace("'", ''))\
                                           .apply(lambda x: x.replace('.', ''))\
                                           .apply(lambda x: synergy_name_dict[x] 
                                                  if x in synergy_name_dict.keys()
                                                  else x)
label_df = label_df.drop('Player', axis=1)
label_df.head()

,season,team,player_cluster,player_alt
0,2017,Memphis Grizzlies,6,marcgasol
1,2017,Memphis Grizzlies,2,tyrekeevans
2,2017,Memphis Grizzlies,1,dillonbrooks
3,2017,Memphis Grizzlies,3,jarellmartin
4,2017,Memphis Grizzlies,1,mariochalmers


Adjust name in the lineup data.

In [23]:
positions = ['PG', 'SG', 'SF', 'PF', 'C']
for pos in positions:
    lineup_df[f'{pos}_alt'] = lineup_df[pos].apply(lambda x: x.replace(' ', ''))\
                                            .apply(lambda x: x.replace("'", ''))\
                                            .apply(lambda x: x.replace('.', ''))\
                                            .apply(lambda x: x.lower())
lineup_df.head()

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt
0,Dennis Schroder,Kent Bazemore,Taurean Prince,Ersan Ilyasova,Miles Plumlee,2017,Atlanta Hawks,733,18.0,99.9,41.0,51.5%,12.0,17.7%,59.0,25.9%,37.0,15.0,dennisschroder,kentbazemore,taureanprince,ersanilyasova,milesplumlee
1,Dennis Schroder,Kent Bazemore,Taurean Prince,John Collins,Dewayne Dedmon,2017,Atlanta Hawks,245,66.0,112.7,81.0,57.9%,15.0,17.1%,37.0,23.1%,41.0,15.7,dennisschroder,kentbazemore,taureanprince,johncollins,dewaynededmon
2,Dennis Schroder,Kent Bazemore,Taurean Prince,Ersan Ilyasova,Dewayne Dedmon,2017,Atlanta Hawks,193,22.0,101.0,18.0,48.8%,78.0,12.4%,22.0,20.4%,73.0,22.1,dennisschroder,kentbazemore,taureanprince,ersanilyasova,dewaynededmon
3,Dennis Schroder,Kent Bazemore,Taurean Prince,Luke Babbitt,Dewayne Dedmon,2017,Atlanta Hawks,142,62.0,112.0,91.0,59.7%,7.0,19.0%,44.0,23.8%,5.0,8.9,dennisschroder,kentbazemore,taureanprince,lukebabbitt,dewaynededmon
4,Isaiah Taylor,Taurean Prince,Damion Lee,John Collins,Dewayne Dedmon,2017,Atlanta Hawks,140,36.0,105.7,41.0,51.5%,62.0,13.6%,36.0,23.1%,4.0,8.3,isaiahtaylor,taureanprince,damionlee,johncollins,dewaynededmon


First attempt to assign cluster label to lineup data.  
The expectation is, there will be some `nan` (missing players) after the merge.

In [24]:
temp = lineup_df.copy()
for pos in positions:
    temp = temp.merge(label_df.rename(columns={'player_alt': f'{pos}_alt'}), 
                      how='left')\
               .rename(columns={'player_cluster': f'{pos}_cluster'})

Find missing players that appear in lineups that logged 100 possessions.

In [25]:
rows_missing = pd.concat([temp[(temp[f'{pos}_cluster'].isna()) & 
                               (temp['Poss'] >= 100)]
                          for pos in positions])\
                 .drop_duplicates()\
                 .sort_values('Poss', ascending=False)

missing_players_data = []
for idx in rows_missing.index:
    row = rows_missing.loc[idx]
    for pos in positions:
        if row[f'{pos}_cluster'] not in [0, 1, 2, 3, 4, 5, 6]:
            missing_players_data.append([row['season'], 
                                         row['team'],
                                         row[pos],
                                         row[f'{pos}_alt']])

missing_players_df = pd.DataFrame(missing_players_data,
                                  columns=['season', 'team', 'player', 'player_alt'])\
                       .drop_duplicates().reset_index(drop=True)
missing_players_df

,season,team,player,player_alt
0,2017,Oklahoma City Thunder,Andre Roberson,andreroberson
1,2017,Oklahoma City Thunder,Corey Brewer,coreybrewer
2,2020,Denver Nuggets,Gary Harris,garyharris
3,2017,Oklahoma City Thunder,Terrance Ferguson,terranceferguson
4,2019,Portland Trail Blazers,Trevor Ariza,trevorariza
...,...,...,...,...
174,2020,Minnesota Timberwolves,Ed Davis,eddavis
175,2020,New York Knicks,Austin Rivers,austinrivers
176,2021,Brooklyn Nets,Day'Ron Sharpe,dayronsharpe
177,2019,Orlando Magic,Al-Farouq Aminu,al-farouqaminu


Find out how many players are missing.

In [26]:
missing_players_df.shape

(179, 4)

In [27]:
s_df['player_alt'] = s_df['Player'].apply(lambda x: x.replace(' ', '').lower())\
                                   .apply(lambda x: x.replace("'", ''))\
                                   .apply(lambda x: x.replace('.', ''))\
                                   .apply(lambda x: synergy_name_dict[x] 
                                          if x in synergy_name_dict.keys()
                                          else x)

Merge Synergy records to missing players.

In [28]:
missing_players_df = missing_players_df.merge(s_df,
                                              left_on=['team', 'season', 'player_alt'],
                                              right_on=['Team', 'Season', 'player_alt'],
                                              how='left')

Make sure that we find all the Synergy data for the missing players.

In [29]:
missing_players_df[missing_players_df['Overall_Poss'].isna()]\
[['Player', 'Team', 'Season', 'Overall_Poss']]

,Player,Team,Season,Overall_Poss


### Step 2: Create cluster tags for those who don't have one

Assign player clusters using previously-trained model.

In [30]:
X = missing_players_df[poss_weights_cols].copy()
X_transformed = pre_cluster_scaler.transform(pca.transform(pre_pca_pipe.transform(X)))
missing_players_df['player_cluster'] = km.predict(X_transformed)
missing_players_df['player_cluster'] = missing_players_df['player_cluster'].map(cluster_map_km_to_sc)

Check players got assigned to cluster 2 or 6, which are relatively primary role.

In [31]:
cluster_dict_detail

{2: 'Ballhandler (Iso+PnR)',
 1: 'Ballhandler (PnR)',
 6: 'Big (Post Up+Roll)',
 5: 'Big (Roll+Cut)',
 0: 'Wing (Off Screen)',
 4: 'Wing (Hand Off)',
 3: 'Wing (Spot Up+Transition)'}

In [32]:
missing_players_df[missing_players_df['Overall_Poss'] < 100]\
.groupby('player_cluster')['player'].count()

player_cluster
0     4
1     4
2     1
3     8
4     3
5    11
6     4
Name: player, dtype: int64

In [33]:
missing_players_df[(missing_players_df['Overall_Poss'] < 100) &
                   (missing_players_df['player_cluster'].isin([2, 6]))]\
[['Player', 'Team', 'Season', 'Overall_Poss', 'player_cluster']].sort_values('Overall_Poss')

,Player,Team,Season,Overall_Poss,player_cluster
153,Dakari Johnson,Oklahoma City Thunder,2017,47.0,6
59,Iman Shumpert,Cleveland Cavaliers,2017,75.0,2
91,Frank Kaminsky,Phoenix Suns,2021,77.0,6
53,Zach Collins,Portland Trail Blazers,2019,79.0,6
19,Dwight Howard,Washington Wizards,2018,95.0,6


In [34]:
# player_lookup('Iman Shumpert', missing_players_df, season=2017)

### Step 3: Complete assigning cluster labels in lineup data

Add cluster labels of missing players to the original label data.

In [35]:
label_df_adj = pd.concat([label_df,
                          missing_players_df[['season', 'team', 
                                              'player_cluster', 'player_alt']]])\
                 .reset_index(drop=True)
label_df_adj.shape

(1953, 4)

Assign labels.

In [36]:
for pos in positions:
    lineup_df = lineup_df.merge(label_df_adj.rename(columns={'player_alt': f'{pos}_alt'}),
                                how='left')\
                         .rename(columns={'player_cluster': f'{pos}_cluster'})

Check if there are missing records with no cluster labels.

In [37]:
pd.concat([lineup_df[(lineup_df[f'{pos}_cluster'].isna()) & 
                     (lineup_df['Poss'] >= 100)]
           for pos in positions])\
  .drop_duplicates()\
  .sort_values('Poss', ascending=False)

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt,PG_cluster,SG_cluster,SF_cluster,PF_cluster,C_cluster


### Step 4: Create cluster embeddings

In [38]:
lineup_df_100 = lineup_df[lineup_df['Poss'] >= 100].reset_index(drop=True)

In [39]:
lineup_df_100['lineup_clusters'] = lineup_df_100.apply(lambda x: [x[f'{pos}_cluster'] 
                                                                  for pos in positions], 
                                                       axis=1)

In [40]:
for i in range(7):
    lineup_df_100[i] = lineup_df_100['lineup_clusters'].apply(lambda x: x.count(i))

In [41]:
lineup_df_100['lineup_embedding'] = lineup_df_100.apply(lambda x: '-'.join([str(x[i]) 
                                                                            for i in range(7)]), 
                                                        axis=1)

### Step 5: Compute 'adjusted offensive ratings'

Inspired by 'NBA Lineup Analysis on Clustered Player Tendencies: A new approach to the positions of basketball & modeling lineup efficiency', the adjusted offensive ratings for lineups that logged under 300 possession are computed like the following:  

$Adj. Off Rating = (\frac{Poss}{300}) * Off Rating + (\frac{300-Poss}{300}) * Tm Off Rating$

In [42]:
lineup_df_100.shape, lineup_df_100[lineup_df_100['Poss'] >= 300].shape

((1521, 37), (302, 37))

In [43]:
lineup_df_100 = lineup_df_100.merge(tm_off_ratings, how='left')

In [44]:
lineup_df_100['adj_off_Pts/Poss'] = lineup_df_100['off_Pts/Poss']
cond = lineup_df_100['Poss'] < 300
lineup_df_100.loc[cond, 'adj_off_Pts/Poss'] = lineup_df_100.loc[cond, 'off_Pts/Poss'] * \
                                              (lineup_df_100.loc[cond, 'Poss'] / 300) + \
                                              lineup_df_100.loc[cond, 'tm_off_Pts/Poss'] * \
                                              ((300 - lineup_df_100.loc[cond, 'Poss']) / 300)

## Exploratory Data Analysis

### Ticket 1: How many types of lineups?

In [45]:
lineup_df_100.groupby('lineup_embedding')[['lineup_clusters']].count().shape

(175, 1)

### Ticket 2: What are the top 10 most popular lineups?

In [46]:
cluster_dict_detail

{2: 'Ballhandler (Iso+PnR)',
 1: 'Ballhandler (PnR)',
 6: 'Big (Post Up+Roll)',
 5: 'Big (Roll+Cut)',
 0: 'Wing (Off Screen)',
 4: 'Wing (Hand Off)',
 3: 'Wing (Spot Up+Transition)'}

In [47]:
def lineup_embedding_to_detail(s, cluster_dict_detail=cluster_dict_detail):
    detail = []
    lineup = s.split('-')
    for i in range(7):
        n = int(lineup[i])
        if n != 0:
            detail.append(f'{n} {cluster_dict_detail[i]}')
    return ' | '.join(detail)

In [48]:
t_2 = lineup_df_100.groupby('lineup_embedding')[['lineup_clusters']]\
                   .count().reset_index()\
                   .rename(columns={'lineup_clusters': 'num_lineups'})
t_2['lineup_detail'] = t_2['lineup_embedding'].apply(lineup_embedding_to_detail)
t_2.sort_values('num_lineups', ascending=False).head(10)

,lineup_embedding,num_lineups,lineup_detail
81,0-2-1-1-0-1-0,57,2 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
132,1-1-1-1-0-1-0,54,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
52,0-1-1-2-0-0-1,51,1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll)
73,0-2-0-2-0-0-1,49,2 Ballhandler (PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll)
126,1-1-0-2-0-1-0,41,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
74,0-2-0-2-0-1-0,39,2 Ballhandler (PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
144,1-2-0-1-0-1-0,38,1 Wing (Off Screen) | 2 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
156,2-0-1-1-0-1-0,32,2 Wing (Off Screen) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
53,0-1-1-2-0-1-0,32,1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut)
131,1-1-1-1-0-0-1,29,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll)


In [49]:
lineup_df_100[lineup_df_100['lineup_embedding'] == '0-2-1-1-0-1-0'].head(1)

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt,PG_cluster,SG_cluster,SF_cluster,PF_cluster,C_cluster,lineup_clusters,0,1,2,3,4,5,6,lineup_embedding,tm_off_Pts/Poss,adj_off_Pts/Poss
72,Kemba Walker,Jaylen Brown,Gordon Hayward,Jayson Tatum,Daniel Theis,2019,Boston Celtics,543,69.0,116.6,79.0,58.1%,63.0,13.1%,36.0,22.9%,23.0,13.8,kembawalker,jaylenbrown,gordonhayward,jaysontatum,danieltheis,1.0,3.0,1.0,2.0,5.0,"[1.0, 3.0, 1.0, 2.0, 5.0]",0,2,1,1,0,1,0,0-2-1-1-0-1-0,113.556036,116.6


In [50]:
t_2.shape, t_2[t_2['num_lineups'] >= 10].shape, t_2[t_2['num_lineups'] >= 25].shape

((175, 3), (47, 3), (16, 3))

### Ticket 3: What are the most/least productive lineups?

Find 10 lineup types with the best offensive ratings

In [51]:
t_3 = lineup_df_100.groupby('lineup_embedding')['adj_off_Pts/Poss'].mean().reset_index()
t_3['lineup_detail'] = t_3['lineup_embedding'].apply(lineup_embedding_to_detail)
t_3 = t_3.merge(t_2, how='left')
t_3.sort_values('adj_off_Pts/Poss', ascending=False).head(10)

,lineup_embedding,adj_off_Pts/Poss,lineup_detail,num_lineups
118,1-0-3-1-0-0-0,128.251674,1 Wing (Off Screen) | 3 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition),1
104,1-0-1-0-0-1-2,123.505404,1 Wing (Off Screen) | 1 Ballhandler (Iso+PnR) | 1 Big (Roll+Cut) | 2 Big (Post Up+Roll),1
24,0-0-3-0-0-1-1,122.064887,3 Ballhandler (Iso+PnR) | 1 Big (Roll+Cut) | 1 Big (Post Up+Roll),1
18,0-0-2-1-1-1-0,121.540182,2 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Wing (Hand Off) | 1 Big (Roll+Cut),1
97,0-4-0-1-0-0-0,121.300000,4 Ballhandler (PnR) | 1 Wing (Spot Up+Transition),1
4,0-0-1-1-1-0-2,120.081548,1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Wing (Hand Off) | 2 Big (Post Up+Roll),2
86,0-2-2-1-0-0-0,119.218687,2 Ballhandler (PnR) | 2 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition),2
158,2-0-2-0-0-0-1,119.185287,2 Wing (Off Screen) | 2 Ballhandler (Iso+PnR) | 1 Big (Post Up+Roll),4
14,0-0-2-1-0-0-2,118.395040,2 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 2 Big (Post Up+Roll),5
64,0-1-3-0-0-0-1,117.494588,1 Ballhandler (PnR) | 3 Ballhandler (Iso+PnR) | 1 Big (Post Up+Roll),3


In [52]:
lineup_df_100[lineup_df_100['lineup_embedding'] == '1-0-3-1-0-0-0']

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt,PG_cluster,SG_cluster,SF_cluster,PF_cluster,C_cluster,lineup_clusters,0,1,2,3,4,5,6,lineup_embedding,tm_off_Pts/Poss,adj_off_Pts/Poss
129,James Harden,Kyrie Irving,Joe Harris,Kevin Durant,Jeff Green,2020,Brooklyn Nets,150,97.0,137.3,97.0,64.9%,45.0,14.0%,24.0,20.6%,64.0,19.6,jamesharden,kyrieirving,joeharris,kevindurant,jeffgreen,2.0,2.0,0.0,2.0,3.0,"[2.0, 2.0, 0.0, 2.0, 3.0]",1,0,3,1,0,0,0,1-0-3-1-0-0-0,119.203348,128.251674


Find 10 common lineup types (at least 10 lineups) with the best offensive ratings

In [53]:
t_3[t_3['num_lineups'] >= 10].sort_values('adj_off_Pts/Poss', ascending=False)\
                             .head(10)

,lineup_embedding,adj_off_Pts/Poss,lineup_detail,num_lineups
114,1-0-2-1-0-1-0,115.359432,1 Wing (Off Screen) | 2 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),22
63,0-1-2-2-0-0-0,114.835165,1 Ballhandler (PnR) | 2 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition),12
23,0-0-2-3-0-0-0,114.683602,2 Ballhandler (Iso+PnR) | 3 Wing (Spot Up+Transition),14
108,1-0-1-2-0-1-0,114.677826,1 Wing (Off Screen) | 1 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),28
52,0-1-1-2-0-0-1,114.243787,1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll),51
131,1-1-1-1-0-0-1,114.139252,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll),29
72,0-2-0-1-1-1-0,114.054107,2 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Wing (Hand Off) | 1 Big (Roll+Cut),15
21,0-0-2-2-0-1-0,113.941615,2 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),21
61,0-1-2-1-0-1-0,113.645206,1 Ballhandler (PnR) | 2 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),19
132,1-1-1-1-0-1-0,113.263513,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),54


In [54]:
lineup_df_100[lineup_df_100['lineup_embedding'] == '1-0-2-1-0-1-0']\
.sort_values('adj_off_Pts/Poss', ascending=False).head(1)

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt,PG_cluster,SG_cluster,SF_cluster,PF_cluster,C_cluster,lineup_clusters,0,1,2,3,4,5,6,lineup_embedding,tm_off_Pts/Poss,adj_off_Pts/Poss
1250,Damian Lillard,CJ McCollum,Norman Powell,Robert Covington,Enes Kanter,2020,Portland Trail Blazers,168,85.0,125.6,36.0,53.2%,74.0,11.9%,91.0,32.6%,93.0,28.0,damianlillard,cjmccollum,normanpowell,robertcovington,eneskanter,2.0,2.0,0.0,3.0,5.0,"[2.0, 2.0, 0.0, 3.0, 5.0]",1,0,2,1,0,1,0,1-0-2-1-0-1-0,119.060365,122.72256


Find 10 common lineup types with worst offensive ratings

In [55]:
t_3[t_3['num_lineups'] >= 10].sort_values('adj_off_Pts/Poss').head(10)

,lineup_embedding,adj_off_Pts/Poss,lineup_detail,num_lineups
76,0-2-0-3-0-0-0,106.299324,2 Ballhandler (PnR) | 3 Wing (Spot Up+Transition),19
127,1-1-0-3-0-0-0,106.857152,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 3 Wing (Spot Up+Transition),14
164,2-1-0-1-0-1-0,107.326704,2 Wing (Off Screen) | 1 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),15
83,0-2-1-2-0-0-0,108.091558,2 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 2 Wing (Spot Up+Transition),11
47,0-1-1-1-0-0-2,108.222316,1 Ballhandler (PnR) | 1 Ballhandler (Iso+PnR) | 1 Wing (Spot Up+Transition) | 2 Big (Post Up+Roll),10
143,1-2-0-1-0-0-1,108.833384,1 Wing (Off Screen) | 2 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Post Up+Roll),24
92,0-3-0-1-0-1-0,109.470857,3 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),24
126,1-1-0-2-0-1-0,109.493412,1 Wing (Off Screen) | 1 Ballhandler (PnR) | 2 Wing (Spot Up+Transition) | 1 Big (Roll+Cut),41
71,0-2-0-1-1-0-1,109.623583,2 Ballhandler (PnR) | 1 Wing (Spot Up+Transition) | 1 Wing (Hand Off) | 1 Big (Post Up+Roll),12
138,1-2-0-0-0-0-2,109.702516,1 Wing (Off Screen) | 2 Ballhandler (PnR) | 2 Big (Post Up+Roll),21


In [56]:
lineup_df_100[lineup_df_100['lineup_embedding'] == '0-2-0-3-0-0-0']\
.sort_values('adj_off_Pts/Poss').head(1)

,PG,SG,SF,PF,C,season,team,Poss,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,PG_alt,SG_alt,SF_alt,PF_alt,C_alt,PG_cluster,SG_cluster,SF_cluster,PF_cluster,C_cluster,lineup_clusters,0,1,2,3,4,5,6,lineup_embedding,tm_off_Pts/Poss,adj_off_Pts/Poss
1043,Theo Maledon,Hamidou Diallo,Kenrich Williams,Aleksej Pokusevski,Mike Muscala,2020,Oklahoma City Thunder,151,2.0,89.4,3.0,45.0%,10.0,17.9%,77.0,29.9%,1.0,6.4,theomaledon,hamidoudiallo,kenrichwilliams,aleksejpokusevski,mikemuscala,1.0,3.0,3.0,1.0,3.0,"[1.0, 3.0, 3.0, 1.0, 3.0]",0,2,0,3,0,0,0,0-2-0-3-0-0-0,103.74686,96.525607


## Modeling